## Evaluating the suitability filter on WILDS

In [ ]:
import pickle
import random

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

from suitability.filter.suitability_efficient import SuitabilityFilter

# Set seeds for reproducibility
random.seed(32)
np.random.seed(32)

This is the code used to evaluate the suitability filter on WILDS datasets (as presented in the paper). Please note that models were trained using the code in `utils/wilds_models.py` and features and split indices were pre-computed based on the code in `utils/prepare_features_and_splits/`. We include the specific features and split indices used for our experiments in `results/features/` and `results/split_indices/`, so the below can be run without training new WILDS models. 

In [ ]:
def calculate_ece_and_bias(probs, correct, n_bins=10):
    """
    Calculate the Expected Calibration Error (ECE) and Calibration Bias (CB).
    
    Args:
        probs (np.ndarray): Array of predicted probabilities for the positive class, shape (n_samples,).
        correct (np.ndarray): Array of correct binary labels (0 or 1), shape (n_samples,).
        n_bins (int): Number of bins to use for calibration calculation.
        
    Returns:
        tuple: (ECE, CB), where:
            - ECE (float): Expected Calibration Error.
            - CB (float): Calibration Bias (positive = overestimation, negative = underestimation).
    """
    # Define bin edges and initialize variables
    bins = np.linspace(0, 1, n_bins + 1)
    ece = 0
    cb = 0
    
    # Assign probabilities to bins
    bin_indices = np.digitize(probs, bins) - 1  # Map probabilities to bin indices (0 to n_bins-1)
    
    # Calculate ECE and CB
    for i in range(n_bins):
        # Mask for the current bin
        bin_mask = bin_indices == i
        if np.sum(bin_mask) == 0:  # Skip empty bins
            continue
        
        # Bin accuracy and confidence
        bin_accuracy = np.mean(correct[bin_mask])
        bin_confidence = np.mean(probs[bin_mask])
        
        # Bin weight
        bin_weight = np.sum(bin_mask) / len(correct)
        
        # Update ECE and CB
        ece += bin_weight * np.abs(bin_accuracy - bin_confidence)
        # cb += bin_weight * (bin_confidence - bin_accuracy)

    cb = np.mean(probs) - np.mean(correct)
    
    return ece, cb

In [ ]:
root_dir = "/mfsnic/projects/suitability/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

algorithm = "ERM"
model_type = "last"
seed = 0

# Define suitability filter and experiment parameters
classifiers = [
    "logistic_regression"
]  # ["logistic_regression", "svm", "random_forest", "gradient_boosting", "mlp", "decision_tree"]
margins = [0] # [0, 0.005, 0.01, 0.05]
normalize = True
calibrated = True
feature_subsets = [
    # [0],
    # [1],
    # [2],
    # [3],
    # [4],
    # [5],
    # [6],
    # [7],
    # [8],
    # [9],
    # [10],
    # [11],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
]
num_fold_arr = [15]

### FMOW

In [ ]:
data_name = "fmow"

# Load the features
feature_cache_file = (
    f"suitability/results/features/{data_name}_{algorithm}_{model_type}_{seed}.pkl"
)
with open(feature_cache_file, "rb") as f:
    full_feature_dict = pickle.load(f)

#### ID SPLITS

In [ ]:
sf_results = []

id_feature_dict = {}
id_feature_dict["id_val"] = full_feature_dict["id_val"]
id_feature_dict["id_test"] = full_feature_dict["id_test"]

# Load the split indices
split_cache_file = f"suitability/results/split_indices/{data_name}_id.pkl"
with open(split_cache_file, "rb") as f:
    id_split_dict = pickle.load(f)

# Main loop
for user_split_name, user_filter in tqdm(id_split_dict.keys()):
    print(f"Evaluating user split: {user_split_name} with filter {user_filter}")

    # Get user split indices
    user_split_indices = id_split_dict[(user_split_name, user_filter)]

    # Get user split features and correctness
    all_features, all_corr = id_feature_dict[user_split_name]
    user_features = all_features[user_split_indices]
    user_corr = all_corr[user_split_indices]
    user_size = len(user_corr)
    user_acc = np.mean(user_corr)

    # Re-partition remaining data into folds
    remaining_indices = np.setdiff1d(np.arange(len(all_corr)), user_split_indices)
    remaining_features = all_features[remaining_indices]
    remaining_corr = all_corr[remaining_indices]
    if user_split_name == "id_val":
        other_split_name = "id_test"
    elif user_split_name == "id_test":
        other_split_name = "id_val"
    else:
        raise ValueError(f"Invalid split name: {user_split_name}")
    additional_features, additional_corr = id_feature_dict[other_split_name]
    source_features = np.concatenate([remaining_features, additional_features], axis=0)
    source_corr = np.concatenate([remaining_corr, additional_corr], axis=0)

    for num_folds in num_fold_arr:
        source_fold_size = len(source_corr) // num_folds
        indices = np.arange(len(source_corr))
        np.random.shuffle(indices)
        fold_indices = [
            indices[i * source_fold_size : (i + 1) * source_fold_size]
            for i in range(num_folds)
        ]

        for i, reg_indices in enumerate(fold_indices):
            reg_features = source_features[reg_indices]
            reg_corr = source_corr[reg_indices]
            reg_size = len(reg_corr)
            reg_acc = np.mean(reg_corr)

            for j, test_indices in enumerate(fold_indices):
                if i == j:
                    continue
                test_features = source_features[test_indices]
                test_corr = source_corr[test_indices]
                test_size = len(test_corr)
                test_acc = np.mean(test_corr)

                for classifier in classifiers:
                    for feature_subset in feature_subsets:
                        suitability_filter = SuitabilityFilter(
                            test_features,
                            test_corr,
                            reg_features,
                            reg_corr,
                            device,
                            normalize=normalize,
                            feature_subset=feature_subset,
                        )
                        suitability_filter.train_classifier(
                            calibrated=calibrated, classifier=classifier
                        )

                        for margin in margins:
                            # Test suitability filter
                            sf_test = suitability_filter.suitability_test(
                                user_features=user_features, margin=margin, return_predictions=True
                            )
                            p_value = sf_test["p_value"]
                            ground_truth = user_acc >= test_acc - margin

                            pred_user = sf_test["user_predictions"]
                            pred_test = sf_test["test_predictions"]

                            # Calculate ECE and CB
                            ece_user, cb_user = calculate_ece_and_bias(
                                pred_user, user_corr
                            )
                            ece_test, cb_test = calculate_ece_and_bias(
                                pred_test, test_corr
                            )

                            sf_results.append(
                                {
                                    "data_name": data_name,
                                    "algorithm": algorithm,
                                    "seed": seed,
                                    "model_type": model_type,
                                    "normalize": normalize,
                                    "calibrated": calibrated,
                                    "margin": margin,
                                    "reg_fold": i,
                                    "reg_size": reg_size,
                                    "reg_acc": reg_acc,
                                    "test_fold": j,
                                    "test_size": test_size,
                                    "test_acc": test_acc,
                                    "user_split": user_split_name,
                                    "user_filter": user_filter,
                                    "user_size": user_size,
                                    "user_acc": user_acc,
                                    "p_value": p_value,
                                    "ground_truth": ground_truth,
                                    "classifier": classifier,
                                    "feature_subset": feature_subset,
                                    "acc_diff": user_acc - test_acc,
                                    "acc_diff_adjusted": user_acc + margin - test_acc,
                                    "ece_user": ece_user,
                                    "cb_user": cb_user,
                                    "ece_test": ece_test,
                                    "cb_test": cb_test,
                                    "mean_pred_user": np.mean(pred_user),
                                    "mean_pred_test": np.mean(pred_test),
                                    "std_pred_user": np.std(pred_user),
                                    "std_pred_test": np.std(pred_test),
                                }
                            )          


# Save results
sf_evals = pd.DataFrame(sf_results)
sf_evals.to_csv(
    f"suitability/results/sf_evals/{data_name}_{algorithm}_{model_type}_{seed}_id.csv",
    index=False,
)


#### OOD SPLITS

In [ ]:
sf_results = []

# Load the split indices
split_cache_file = f"suitability/results/split_indices/{data_name}_ood.pkl"
with open(split_cache_file, "rb") as f:
    ood_split_dict = pickle.load(f)

id_features_val, id_corr_val = full_feature_dict["id_val"]
id_features_test, id_corr_test = full_feature_dict["id_test"]
source_features = np.concatenate([id_features_val, id_features_test], axis=0)
source_corr = np.concatenate([id_corr_val, id_corr_test], axis=0)


# Main loop
for user_split_name, user_filter in tqdm(ood_split_dict.keys()):
    print(f"Evaluating user split: {user_split_name} with filter {user_filter}")

    # Get user split indices
    user_split_indices = ood_split_dict[(user_split_name, user_filter)]

    # Get user split features and correctness
    all_features, all_corr = full_feature_dict[user_split_name]
    user_features = all_features[user_split_indices]
    user_corr = all_corr[user_split_indices]
    user_size = len(user_corr)
    user_acc = np.mean(user_corr)

    for num_folds in num_fold_arr:
        source_fold_size = len(source_corr) // num_folds
        indices = np.arange(len(source_corr))
        np.random.shuffle(indices)
        fold_indices = [
            indices[i * source_fold_size : (i + 1) * source_fold_size]
            for i in range(num_folds)
        ]

        for i, reg_indices in enumerate(fold_indices):
            reg_features = source_features[reg_indices]
            reg_corr = source_corr[reg_indices]
            reg_size = len(reg_corr)
            reg_acc = np.mean(reg_corr)

            for j, test_indices in enumerate(fold_indices):
                if i == j:
                    continue
                test_features = source_features[test_indices]
                test_corr = source_corr[test_indices]
                test_size = len(test_corr)
                test_acc = np.mean(test_corr)

                for classifier in classifiers:
                    for feature_subset in feature_subsets:
                        suitability_filter = SuitabilityFilter(
                            test_features,
                            test_corr,
                            reg_features,
                            reg_corr,
                            device,
                            normalize=normalize,
                            feature_subset=feature_subset,
                        )
                        suitability_filter.train_classifier(
                            calibrated=calibrated, classifier=classifier
                        )

                        for margin in margins:
                            # Test suitability filter
                            sf_test = suitability_filter.suitability_test(
                                user_features=user_features, margin=margin, return_predictions=True
                            )
                            p_value = sf_test["p_value"]
                            ground_truth = user_acc >= test_acc - margin

                            pred_user = sf_test["user_predictions"]
                            pred_test = sf_test["test_predictions"]

                            # Calculate ECE and CB
                            ece_user, cb_user = calculate_ece_and_bias(
                                pred_user, user_corr
                            )
                            ece_test, cb_test = calculate_ece_and_bias(
                                pred_test, test_corr
                            )

                            sf_results.append(
                                {
                                    "data_name": data_name,
                                    "algorithm": algorithm,
                                    "seed": seed,
                                    "model_type": model_type,
                                    "normalize": normalize,
                                    "calibrated": calibrated,
                                    "margin": margin,
                                    "reg_fold": i,
                                    "reg_size": reg_size,
                                    "reg_acc": reg_acc,
                                    "test_fold": j,
                                    "test_size": test_size,
                                    "test_acc": test_acc,
                                    "user_split": user_split_name,
                                    "user_filter": user_filter,
                                    "user_size": user_size,
                                    "user_acc": user_acc,
                                    "p_value": p_value,
                                    "ground_truth": ground_truth,
                                    "classifier": classifier,
                                    "feature_subset": feature_subset,
                                    "acc_diff": user_acc - test_acc,
                                    "acc_diff_adjusted": user_acc + margin - test_acc,
                                    "ece_user": ece_user,
                                    "cb_user": cb_user,
                                    "ece_test": ece_test,
                                    "cb_test": cb_test,
                                    "mean_pred_user": np.mean(pred_user),
                                    "mean_pred_test": np.mean(pred_test),
                                    "std_pred_user": np.std(pred_user),
                                    "std_pred_test": np.std(pred_test),
                                }
                            )

# Save results
sf_evals = pd.DataFrame(sf_results)
sf_evals.to_csv(
    f"suitability/results/sf_evals/{data_name}_{algorithm}_{model_type}_{seed}_ood.csv",
    index=False,
)


### RXRX1

In [ ]:
data_name = "rxrx1"

# Load the features
feature_cache_file = (
    f"suitability/results/features/{data_name}_{algorithm}_{model_type}_{seed}.pkl"
)
with open(feature_cache_file, "rb") as f:
    full_feature_dict = pickle.load(f)

#### ID SPLITS

In [ ]:
sf_results = []

# Load the split indices
split_cache_file = f"suitability/results/split_indices/{data_name}.pkl"
with open(split_cache_file, "rb") as f:
    split_dict = pickle.load(f)

all_keys = list(split_dict.keys())
for key in all_keys:
    if key[0] == 'test' or key[0] == 'val':
        del split_dict[key]

# Main loop
for user_split_name, user_filter in tqdm(split_dict.keys()):
    print(f"Evaluating user split: {user_split_name} with filter {user_filter}")

    # Get user split indices
    user_split_indices = split_dict[(user_split_name, user_filter)]

    # Get user split features and correctness
    all_features, all_corr = full_feature_dict[user_split_name]
    user_features = all_features[user_split_indices]
    user_corr = all_corr[user_split_indices]
    user_size = len(user_corr)
    user_acc = np.mean(user_corr)

    # Re-partition remaining data into folds
    remaining_indices = np.setdiff1d(np.arange(len(all_corr)), user_split_indices)
    source_features = all_features[remaining_indices]
    source_corr = all_corr[remaining_indices]

    for num_folds in num_fold_arr:
        source_fold_size = len(source_corr) // num_folds
        indices = np.arange(len(source_corr))
        np.random.shuffle(indices)
        fold_indices = [
            indices[i * source_fold_size : (i + 1) * source_fold_size]
            for i in range(num_folds)
        ]

        for i, reg_indices in enumerate(fold_indices):
            reg_features = source_features[reg_indices]
            reg_corr = source_corr[reg_indices]
            reg_size = len(reg_corr)
            reg_acc = np.mean(reg_corr)

            for j, test_indices in enumerate(fold_indices):
                if i == j:
                    continue
                test_features = source_features[test_indices]
                test_corr = source_corr[test_indices]
                test_size = len(test_corr)
                test_acc = np.mean(test_corr)

                for classifier in classifiers:
                    for feature_subset in feature_subsets:
                        suitability_filter = SuitabilityFilter(
                            test_features,
                            test_corr,
                            reg_features,
                            reg_corr,
                            device,
                            normalize=normalize,
                            feature_subset=feature_subset,
                        )
                        suitability_filter.train_classifier(
                            calibrated=calibrated, classifier=classifier
                        )

                        for margin in margins:
                            # Test suitability filter
                            sf_test = suitability_filter.suitability_test(
                                user_features=user_features, margin=margin
                            )
                            p_value = sf_test["p_value"]
                            ground_truth = user_acc >= test_acc - margin

                            sf_results.append(
                                {
                                    "data_name": data_name,
                                    "algorithm": algorithm,
                                    "seed": seed,
                                    "model_type": model_type,
                                    "normalize": normalize,
                                    "calibrated": calibrated,
                                    "margin": margin,
                                    "reg_fold": i,
                                    "reg_size": reg_size,
                                    "reg_acc": reg_acc,
                                    "test_fold": j,
                                    "test_size": test_size,
                                    "test_acc": test_acc,
                                    "user_split": user_split_name,
                                    "user_filter": user_filter,
                                    "user_size": user_size,
                                    "user_acc": user_acc,
                                    "p_value": p_value,
                                    "ground_truth": ground_truth,
                                    "classifier": classifier,
                                    "feature_subset": feature_subset,
                                    "acc_diff": user_acc - test_acc,
                                    "acc_diff_adjusted": user_acc + margin - test_acc,
                                }
                            )

# Save results
sf_evals = pd.DataFrame(sf_results)
sf_evals.to_csv(
    f"suitability/results/sf_evals/erm/{data_name}_{algorithm}_{model_type}_{seed}_id.csv",
    index=False,
)


#### OOD SPLITS

In [ ]:
sf_results = []

# Load the split indices
split_cache_file = f"suitability/results/split_indices/{data_name}.pkl"
with open(split_cache_file, "rb") as f:
    ood_split_dict = pickle.load(f)

all_keys = list(ood_split_dict.keys())
for key in all_keys:
    if key[0] == 'id_test':
        del ood_split_dict[key]

source_features, source_corr = full_feature_dict["id_test"]

# Main loop
for user_split_name, user_filter in tqdm(ood_split_dict.keys()):
    print(f"Evaluating user split: {user_split_name} with filter {user_filter}")

    # Get user split indices
    user_split_indices = ood_split_dict[(user_split_name, user_filter)]

    # Get user split features and correctness
    all_features, all_corr = full_feature_dict[user_split_name]
    user_features = all_features[user_split_indices]
    user_corr = all_corr[user_split_indices]
    user_size = len(user_corr)
    user_acc = np.mean(user_corr)

    for num_folds in num_fold_arr:
        source_fold_size = len(source_corr) // num_folds
        indices = np.arange(len(source_corr))
        np.random.shuffle(indices)
        fold_indices = [
            indices[i * source_fold_size : (i + 1) * source_fold_size]
            for i in range(num_folds)
        ]

        for i, reg_indices in enumerate(fold_indices):
            reg_features = source_features[reg_indices]
            reg_corr = source_corr[reg_indices]
            reg_size = len(reg_corr)
            reg_acc = np.mean(reg_corr)

            for j, test_indices in enumerate(fold_indices):
                if i == j:
                    continue
                test_features = source_features[test_indices]
                test_corr = source_corr[test_indices]
                test_size = len(test_corr)
                test_acc = np.mean(test_corr)

                for classifier in classifiers:
                    for feature_subset in feature_subsets:
                        suitability_filter = SuitabilityFilter(
                            test_features,
                            test_corr,
                            reg_features,
                            reg_corr,
                            device,
                            normalize=normalize,
                            feature_subset=feature_subset,
                        )
                        suitability_filter.train_classifier(
                            calibrated=calibrated, classifier=classifier
                        )

                        for margin in margins:
                            # Test suitability filter
                            sf_test = suitability_filter.suitability_test(
                                user_features=user_features, margin=margin
                            )
                            p_value = sf_test["p_value"]
                            ground_truth = user_acc >= test_acc - margin

                            sf_results.append(
                                {
                                    "data_name": data_name,
                                    "algorithm": algorithm,
                                    "seed": seed,
                                    "model_type": model_type,
                                    "normalize": normalize,
                                    "calibrated": calibrated,
                                    "margin": margin,
                                    "reg_fold": i,
                                    "reg_size": reg_size,
                                    "reg_acc": reg_acc,
                                    "test_fold": j,
                                    "test_size": test_size,
                                    "test_acc": test_acc,
                                    "user_split": user_split_name,
                                    "user_filter": user_filter,
                                    "user_size": user_size,
                                    "user_acc": user_acc,
                                    "p_value": p_value,
                                    "ground_truth": ground_truth,
                                    "classifier": classifier,
                                    "feature_subset": feature_subset,
                                    "acc_diff": user_acc - test_acc,
                                    "acc_diff_adjusted": user_acc + margin - test_acc,
                                }
                            )

# Save results
sf_evals = pd.DataFrame(sf_results)
sf_evals.to_csv(
    f"suitability/results/sf_evals/erm/{data_name}_{algorithm}_{model_type}_{seed}_ood.csv",
    index=False,
)


### CIVILCOMMENTS

In [ ]:
data_name = "civilcomments"

# Load the features
feature_cache_file = (
    f"suitability/results/features/{data_name}_{algorithm}_{model_type}_{seed}.pkl"
)
with open(feature_cache_file, "rb") as f:
    feature_dict = pickle.load(f)

sf_results = []

# Load the split indices
split_cache_file = f"suitability/results/split_indices/{data_name}.pkl"
with open(split_cache_file, "rb") as f:
    split_dict = pickle.load(f)


# Main loop
for user_split_name, user_filter in tqdm(split_dict.keys()):
    print(f"Evaluating user split: {user_split_name} with filter {user_filter}")

    # Get user split indices
    user_split_indices = split_dict[(user_split_name, user_filter)]

    # Get user split features and correctness
    all_features, all_corr = feature_dict[user_split_name]
    user_features = all_features[user_split_indices]
    user_corr = all_corr[user_split_indices]
    user_size = len(user_corr)
    user_acc = np.mean(user_corr)

    # Re-partition remaining data into folds
    remaining_indices = np.setdiff1d(np.arange(len(all_corr)), user_split_indices)
    remaining_features = all_features[remaining_indices]
    remaining_corr = all_corr[remaining_indices]
    if user_split_name == "val":
        other_split_name = "test"
    elif user_split_name == "test":
        other_split_name = "val"
    else:
        raise ValueError(f"Invalid split name: {user_split_name}")
    additional_features, additional_corr = feature_dict[other_split_name]
    source_features = np.concatenate([remaining_features, additional_features], axis=0)
    source_corr = np.concatenate([remaining_corr, additional_corr], axis=0)

    for num_folds in num_fold_arr:
        source_fold_size = len(source_corr) // num_folds
        indices = np.arange(len(source_corr))
        np.random.shuffle(indices)
        fold_indices = [
            indices[i * source_fold_size : (i + 1) * source_fold_size]
            for i in range(num_folds)
        ]

        for i, reg_indices in enumerate(fold_indices):
            reg_features = source_features[reg_indices]
            reg_corr = source_corr[reg_indices]
            reg_size = len(reg_corr)
            reg_acc = np.mean(reg_corr)

            for j, test_indices in enumerate(fold_indices):
                if i == j:
                    continue
                test_features = source_features[test_indices]
                test_corr = source_corr[test_indices]
                test_size = len(test_corr)
                test_acc = np.mean(test_corr)

                for classifier in classifiers:
                    for feature_subset in feature_subsets:
                        suitability_filter = SuitabilityFilter(
                            test_features,
                            test_corr,
                            reg_features,
                            reg_corr,
                            device,
                            normalize=normalize,
                            feature_subset=feature_subset,
                        )
                        suitability_filter.train_classifier(
                            calibrated=calibrated, classifier=classifier
                        )

                        for margin in margins:
                            # Test suitability filter
                            sf_test = suitability_filter.suitability_test(
                                user_features=user_features, margin=margin
                            )
                            p_value = sf_test["p_value"]
                            ground_truth = user_acc >= test_acc - margin

                            sf_results.append(
                                {
                                    "data_name": data_name,
                                    "algorithm": algorithm,
                                    "seed": seed,
                                    "model_type": model_type,
                                    "normalize": normalize,
                                    "calibrated": calibrated,
                                    "margin": margin,
                                    "reg_fold": i,
                                    "reg_size": reg_size,
                                    "reg_acc": reg_acc,
                                    "test_fold": j,
                                    "test_size": test_size,
                                    "test_acc": test_acc,
                                    "user_split": user_split_name,
                                    "user_filter": user_filter,
                                    "user_size": user_size,
                                    "user_acc": user_acc,
                                    "p_value": p_value,
                                    "ground_truth": ground_truth,
                                    "classifier": classifier,
                                    "feature_subset": feature_subset,
                                    "acc_diff": user_acc - test_acc,
                                    "acc_diff_adjusted": user_acc + margin - test_acc,
                                }
                            )


# Save results
sf_evals = pd.DataFrame(sf_results)
sf_evals.to_csv(
    f"suitability/results/sf_evals/erm/{data_name}_{algorithm}_{model_type}_{seed}_.csv",
    index=False,
)
